In [ ]:
!pip install apache-beam[gcp]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.5/173.5 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.9/197.9 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.6/304.6 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.7/402.7 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 7.7 MB/s et

In [ ]:
# Create input.txt in the Colab environment
with open('input.txt', 'w') as f:
    f.write("his exercise is to learn apache beam\n")
    f.write("I will understand apache beam after this assignment\n")
    f.write("Lets find out what apache beam is\n")


In [ ]:
import apache_beam as beam
from apache_beam.transforms.window import FixedWindows
from apache_beam.options.pipeline_options import PipelineOptions
import time

# Define a custom composite transform (word count)
class CountWords(beam.PTransform):
    def expand(self, pcoll):
        return (
            pcoll
            | "Split words" >> beam.FlatMap(lambda x: x.split())
            | "Pair with 1" >> beam.Map(lambda x: (x, 1))
            | "Group and sum" >> beam.CombinePerKey(sum)
        )

# Create a pipeline
options = PipelineOptions()

with beam.Pipeline(options=options) as p:
    # Step 1: Reading from an input text file
    lines = p | "Read Input" >> beam.io.ReadFromText('input.txt')

    # Step 2: Use the custom composite transform for word counting
    word_counts = lines | "Count words" >> CountWords()

    # Step 3: Windowing
    windowed_counts = (
        word_counts
        | "Apply Fixed Window" >> beam.WindowInto(FixedWindows(60))  # 60-second windows
        | "Sum counts in window" >> beam.CombinePerKey(sum)
    )

    # Step 4: Triggers
    windowed_counts = (
        windowed_counts
        | "Trigger after processing" >> beam.WindowInto(
            FixedWindows(60),
            trigger=beam.transforms.trigger.AfterProcessingTime(10),  # 10 seconds
            accumulation_mode=beam.transforms.trigger.AccumulationMode.DISCARDING
        )
    )

    # Step 5: ParDo
    class ProcessWords(beam.DoFn):
        def process(self, element):
            word, count = element
            yield f"Word: {word}, Count: {count}"

    processed = word_counts | "Process Words with ParDo" >> beam.ParDo(ProcessWords())

    # Step 6: Write output to text file Streaming
    processed | "Write Output" >> beam.io.WriteToText('output.txt')



In [ ]:
!cat output.txt-00000-of-00001


Word: his, Count: 1
Word: exercise, Count: 1
Word: is, Count: 2
Word: to, Count: 1
Word: learn, Count: 1
Word: apache, Count: 3
Word: beam, Count: 3
Word: I, Count: 1
Word: will, Count: 1
Word: understand, Count: 1
Word: after, Count: 1
Word: this, Count: 1
Word: assignment, Count: 1
Word: Lets, Count: 1
Word: find, Count: 1
Word: out, Count: 1
Word: what, Count: 1
